In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import sncosmo
import json

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
username = 'jnordin'
date = '2018-09-24'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
                    # At this stage we will also cut history older than this in fit
SNcut = 5           # Need at least some photometry with this Signal to Noise

marshal_savid = 42
logpath = '/home/jnordin/local/github/AmpelSlackCosmology/logs/'

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sne_following.json', 'r') as fp:
    sne_following = json.load(fp)

# log
logger = logging.getLogger(username)
handler = logging.FileHandler(logpath+'ztfcosmo_trigger_%s_%s.log'%(date,username))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

manual_inspection=[]

In [3]:
# Now fun starts for real. 
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

DEBUG:marshaltools.ProgramList:listing accessible programs
DEBUG:marshaltools.ProgramList:Starting list_programs.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_programs.cgi?None
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
DEBUG:marshaltools.ProgramList:Starting list_program_sources.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_program_sources.cgi?programidx=4&getredshift=1&getclassification=1
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Loaded 192 saved sources for program AMPEL Test.


In [4]:
# This is the timeconsuming part. Downloading all the data and do saltfits. Let it start and get a coffee
for count, sn in enumerate(pl.sources.keys()):
    if not pl.sources[sn]['classification'] is None:
        logger.info("SN %s classified with type %s, skipping."%(sn,pl.sources[sn]['classification']))
        continue

    if 'SNCOSMOFIT' in pl.sources[sn]:
        print('already done')
        continue
        
    # Download specific LC
    lc = pl.get_lightcurve(sn)

    # Check nbr filters filters
    if len(np.unique(lc.table["filter"][lc.table["magpsf"]<99] ))<2:
        logger.info("SN %s has too few filters for saltfit."%(sn))
        continue
    
    
    # create a model
    model = sncosmo.Model(source='salt2')
    
    # Massage data
    data = lc.table_sncosmo
    data["mjd"] = data["mjd"]-Time.now().mjd
    data = data[ (data["mjd"]>-maxage) ]
    
    # Replicating data quality checks
    signalNoise = np.abs(data["flux"])/data["fluxerr"]
    data = data[ (signalNoise>SNcut) ]
    
    
    
    # run the fit
    if len(signalNoise[signalNoise>SNcut])==0:
        manual_inspection.append(sn)
        logger.info("Not enugh good data for SN %s. Manual inspection prob needed."%(sn))
        continue    
        
    try:
        result, fitted_model = sncosmo.fit_lc(
            data, model,
            ['z', 't0', 'x0', 'x1', 'c'],  # parameters of model to vary
            bounds={'z':(0., 0.2),'x1':(-5.,5.),'t0':(-20.,15.),'c':(-1.,10.)})  # bounds on parameters (if any)
        pl.sources[sn]['SNCOSMOfit'] = result, fitted_model, data
#        sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
#        plt.show()

            
    except ValueError:
        print("Why does fit fail for %s?"%(sn))
        manual_inspection.append(sn)
        logger.info("SALT fit fails for SN %s. Manual inspection prob needed."%(sn))
        continue

        
        

DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18aakgsmi
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:SN ZTF18aasdted classified with type SN Ia, skipping.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18aaupjtd
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:SN ZTF18aauvptt classified with type AGN, skipping.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abmwzws
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skippe

INFO:jnordin:Not enugh good data for SN ZTF18abmadcc. Manual inspection prob needed.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18ablrwlk
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:Not enugh good data for SN ZTF18ablrwlk. Manual inspection prob needed.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abmdfba
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:Not enugh good data for SN ZTF18abmdfba. Manual inspection prob needed.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abmorft
DEBUG:marshaltools.gci_utils:Successful growth connection.
I

DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abmkbqo
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:Not enugh good data for SN ZTF18abmkbqo. Manual inspection prob needed.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abnnnzk
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abotaki
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:SN ZTF18abotaki has too few filters for saltfit.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZT

INFO:jnordin:SN ZTF18abssukb classified with type SN Ia, skipping.
INFO:jnordin:SN ZTF18absdjms classified with type SN Ia, skipping.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18ablhsgt
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abnorzb
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abottbj
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skip

DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abwlonm
DEBUG:marshaltools.gci_utils:Successful growth connection.
INFO:jnordin:SN ZTF18abwlonm has too few filters for saltfit.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abwblyl
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abwqgsc
DEBUG:marshaltools.gci_utils:Successful growth connection.
DEBUG:marshaltools.gci_utils:Starting print_lc.cgi post. Attempt # 0
DEBUG:marshaltools.gci_utils:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/print_lc.cgi?name=ZTF18abwbaim
DEBUG

In [7]:
visual_inspect = []
for sn in pl.sources.keys():
    if not 'SNCOSMOfit' in pl.sources[sn].keys():
        continue
    visual_inspect.append(sn)
print("Found %s lightcurves to inspect"%(len(visual_inspect)))     



Found 76 lightcurves to inspect


In [6]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Flash SNCOSMO fit plot
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = visual_inspect[snnbr]
    if i == 'TooLate':
        logger.info("%s Faded too much while waiting. Call P200 or Keck?"%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'VeryGood':
        logger.info("Nice looking likely SN %s."%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Possibly':
        logger.info("Get SN %s if we can."%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'NoSNIa':
        logger.info("Object %s most likely not SNIa. "%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'BadData':
        logger.info("Too little data of %s, skip. "%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(visual_inspect):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    
    # Reset    
    snname = visual_inspect[snnbr]
    
    # Retrieve saltfit
    result, fitted_model, data = pl.sources[snname]['SNCOSMOfit']  
    sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)


wiggy = widgets.RadioButtons(
    options=['Nothing','TooLate', 'VeryGood', 'Possibly','NoSNIa','BadData','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [8]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [9]:
# This is the all important list of decisions you have made
decisions = {}

In [10]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

interactive(children=(RadioButtons(description='Action:', options=('Nothing', 'TooLate', 'VeryGood', 'Possibly…

In [11]:
# Go through current list of SNe on the obs queue. If already 
for snnbr, choice in decisions.items():
    if not (choice=='VeryGood' or choice=='Possibly'):
        continue
    sn = visual_inspect[snnbr]
    print("%s %s"%(sn,choice))
    if sn in sne_following.keys():
        logger.info("%s already in obs queue. Adding with setting %s."%(sn,choice))
        sne_following[sn].append([choice,username,date])
    else:
        logger.info("Adding %s to obs queue with setting %s."%(sn,choice))
        sne_following[sn] = [ [choice,username,date] ]

INFO:jnordin:Adding ZTF18ablllyw to obs queue with setting VeryGood.
INFO:jnordin:Adding ZTF18abthaii to obs queue with setting Possibly.
INFO:jnordin:Adding ZTF18abtghbi to obs queue with setting VeryGood.
INFO:jnordin:Adding ZTF18abtswjk to obs queue with setting Possibly.
INFO:jnordin:Adding ZTF18abqdwze to obs queue with setting Possibly.
INFO:jnordin:Adding ZTF18abwfmot to obs queue with setting Possibly.
INFO:jnordin:Adding ZTF18absoomk to obs queue with setting VeryGood.
INFO:jnordin:Adding ZTF18abvnwzz to obs queue with setting Possibly.


ZTF18ablllyw VeryGood
ZTF18abthaii Possibly
ZTF18abtghbi VeryGood
ZTF18abtswjk Possibly
ZTF18abqdwze Possibly
ZTF18abwfmot Possibly
ZTF18absoomk VeryGood
ZTF18abvnwzz Possibly


In [12]:
# Unfortunately not done yet. First check whether any of our SNe to obs are bright enough for RCF.
for snname in sne_following.keys():
    
    lc = pl.get_lightcurve(snname)
    peakmag = np.min(lc.table['magpsf'])
    recentmag =  np.mean(lc.table['magpsf'][ (lc.table['jdobs']-Time.now().jd)>-5 ] )
    
    lastchoice = sne_following[snname][-1][0]
    
    if peakmag<18.7 and recentmag < 20:
        logger.info("%s could be RCF target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
        msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
        print( "Follow link and trigger (RCF) SEDM obs with prio %s"%(lastchoice) )
        print(msg)
    elif peakmag<19.5 and recentmag < 20:
        logger.info("%s could be Cosmology SEDM target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
        msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
        print( "Follow link and trigger (Cosmologz) SEDM obs with prio %s"%(lastchoice) )
        print(msg)
    


/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2954: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakmag 19.28 recentmag 19.84.
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakmag 18.9 recentmag 19.7275.
INFO:jnordin:ZTF18abvnwzz could be RCF target? peakmag 18.27 recentmag 19.231250000000003.
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakmag 19.2 recentmag 19.41.
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakmag 19.39 recentmag 19.93.
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakmag 18.76 recentmag 18.964000000000002.
INFO:jnordin:ZTF18abvnwzz could be Cosmology SEDM target? peakm

Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abrwswl
Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abptskn
Follow link and trigger (RCF) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abscygs
Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18absnsqx
Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abrwrch
Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abtogdl
Follow link and trigger (Cosmologz) SEDM obs with prio VeryGood
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF1

In [14]:
# Now we have to go back to the sne selected for manual verification below:
# Trigger observations of these and add name below
for sn in manual_inspection:
    msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(sn)
    print( "Suggested have a manual look at %s"%(sn) )
    print(msg)
    

Suggested have a manual look at ZTF18ablviog
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18ablviog
Suggested have a manual look at ZTF18abmjayl
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abmjayl
Suggested have a manual look at ZTF18abmszpd
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abmszpd
Suggested have a manual look at ZTF18abmadcc
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abmadcc
Suggested have a manual look at ZTF18ablrwlk
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18ablrwlk
Suggested have a manual look at ZTF18abmdfba
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abmdfba
Suggested have a manual look at ZTF18abmorft
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abmorft
Suggested have a manual look at ZTF18abmpbjy
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18

In [15]:
# Save the list of SN w triggered obs
with open('sne_following.json', 'w') as fp:
    json.dump(sne_following, fp)


In [16]:
# Enter the names of the SNe you triggered SEDM obs for
sedmtrig = []
logger.info("Triggered SEDM obs of: %s"%(sedmtrig))

INFO:jnordin:Triggered SEDM obs of: []
